In [1]:
%load_ext autoreload
%autoreload 2
#from lib.multitasking_lib import *

In [2]:
from lib.multitasking_lib import *
cache_type ='backtest_uni'

my_cache = cache_state('backtest_uni')

In [ ]:
######################
# Initiate Backtest  #
######################
def store_algo(algo, algo_name="default"):
    redis_conn.hset("algos",algo_name, algo)
    algo_f = open("algo/"+algo_name+".txt", "w")
    algo_f.write(algo)
    algo_f.close()
    
#my_cache.flushall()
def backtest_unitest(stocks, freq='1T',algo_name = 'haikin_1_new', fromDate='2019-4-8', toDate='2019-4-9', mode='full'):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    backtest_msg={'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq, 'mode':mode}

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
my_cache.set('done'+cache_type,1)
my_cache.xtrim('msgBufferQueue'+cache_type,0,False)
my_cache.set('last_id_msg'+cache_type, 0)


try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler","order_handler"], [kite_simulator, ohlc_tick_handler, order_handler])

#redis_conn.publish('ohlc_tick_handler'+cache_type,'start')
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


In [ ]:
backtest_unitest(['TCS'])

In [ ]:
################################
###         Stop Trade       ###
################################
pinfo(cache_type)
my_cache.xtrim('msgBufferQueue'+cache_type,0,False)
my_cache.delete('msgBufferQueue'+cache_type)
my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())

if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
time.sleep(1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
my_cache.set('last_id_msg',0)
jobs
jobs = []

In [ ]:
pd.read_json(my_cache.get('TCSbacktest_uniTrade'))

In [ ]:
redis_conn.pubsub_channels()

In [ ]:
backtest_unitest(['TCS'])

In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [ ]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
chart_type ='haikin'

symbol='TCS'
my_cache = cache_state(cache_type)

#######  Hack for freq  ############
my_cache.setValue('TCS','freq','1T')
#####################################
dfohlc = my_cache.getOHLC(symbol)
print(dfohlc.tail())
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(dfohlc, trade_df, symbol, chart_type)

In [ ]:
cache_state('backtest_uni').getValue()

## Live Trade Debugging

In [ ]:
live = threadManager('live', ["ohlc_tick_handler"], [ohlc_tick_handler])


In [ ]:
live_cache.setValue('WIPRO','mode','paper')

In [ ]:
live.job.terminate()
live_cache.set('WIPROliveTICK', pd.DataFrame().to_json(orient='columns'))
live_cache.setValue('WIPRO','last_processed',0)
live_cache.getValue('WIPRO')

In [ ]:
#live_cache.getOHLC('WIPRO') 
print(live_cache.xrange('msgBufferQueuelive')[-2:])
print(datetime.fromtimestamp(int(live_cache.xrange('msgBufferQueuelive')[-1][0].split('-')[0] )/1000))


In [ ]:
live_cache.getOHLC('WIPRO').tail()

## Quick Backtest

In [ ]:
###########################################################
###                  Quick Backtest                     ###
###########################################################
from talib import MACD, MACDEXT, RSI, BBANDS, MACD, AROON, STOCHF, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX, EMA, SMA
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC, MIN, MAX, MINMAX

ohlc_get = lambda df, key: df.iloc[-1][key]
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([LOW,OPEN,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)


################ Load Data ###################
from lib.charting_lib import *
cache_type = 'backtest_uni'
symbol='TCS'
my_cache = cache_state(cache_type)

###  Set Freq for Algo
my_cache.setValue('TCS','freq','1T')
dfohlc = my_cache.getOHLC(symbol)
#print(dfohlc.head())

### Load Data
ohlc_data_temp = dfohlc
pd.set_option('display.max_rows', None)
################ Initialize Helper Functions ###################
trade_df1 = pd.DataFrame()
OPEN = ohlc_data_temp['open']
CLOSE = ohlc_data_temp['close']
HIGH = ohlc_data_temp['high']
LOW = ohlc_data_temp['low']
#VOLUME = ohlc_data_temp['volume']

(haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)

TIME = ohlc_data_temp.index[-1].minute+ohlc_data_temp.index[-1].hour*60

#REF = lambda df, i: df.iloc[-i-1]
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)


def BUY(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["buy"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1

def SELL(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["sell"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1
    
################# Write Algo For Backtest Here ###################

#sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
#print(trade_df1)
sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) < REF(haCLOSE,1))
buy = (REF(haOPEN, 0) < REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))

logic = buy | sell

summary_df = trade_df1.copy(deep=True)
#summary_df['haopen'] = haOPEN[ logic ]
#summary_df['haclose'] = haCLOSE[ logic ]
summary_df['roc'] = ROC(CLOSE, 30)[logic]

#print(summary_df.sort_index())
#print(summary_df.sort_index().tail(10))
#print(ROC(CLOSE,3))
#################### Render Charts ###############################

trade_df1 = SELL(CLOSE, sell, trade_df1)
trade_df1 = BUY(CLOSE, buy, trade_df1)

#print(trade_df1.sort_index())
my_cache.setCache(symbol+cache_type+'Trade',trade_df1)
#my_cache.pushTrade(symbol, trade_df1)

chart_type ='haikin'
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(ohlc_data_temp, trade_df, symbol, chart_type)

In [ ]:
cache_type = 'backtest_web'
symbol='TCS'
my_cache = cache_state(cache_type)
my_cache.getTrades(symbol)

my_cache.get('TCSbacktest_webTrades')

In [ ]:
my_cache.getValue()

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

In [ ]:
live_cache.hget('algos','SUHAN') is not None

## Redis Console

In [3]:
from lib.multitasking_lib import *
#from lib.data_model_lib import *
import json
web_cache = cache_state('backtest_web')
live_cache = cache_state('live')
uni_cache = cache_state('backtest_uni')

pd.set_option('display.max_rows', None)

In [ ]:
live_cache.pubsub_channels()

In [4]:
live_cache.xread({'notificationQueuelivenew':0}, block=2000, count=5000)

[]

In [ ]:
values = ['TCS','WIPRO','CIPLA','AXISBANK']
df_cache = live_cache.getValue()
cache_keys =  live_cache.getKeys()

if values is not None:
    for value in values:
        if value not in live_cache.getKeys():
            print(value)

In [ ]:
live_cache.remove()

In [ ]:
df = uni_cache.getValue()
df

In [ ]:
live_cache.getOHLC('TCS')

In [ ]:
trades_df = live_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

### GetData

In [ ]:
logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [ ]:
ohlc_data[stock].head()

# Kite Connect

In [ ]:
from kiteconnect import KiteConnect
kite_api_key = 'b2w0sfnr1zr92nxm'
live_cache.set('KiteAPIKey',kite_api_key)
access_token = live_cache.get('access_token')
print(access_token)
print(live_cache.get('KiteAPIKey'))

kite = KiteConnect(api_key=kite_api_key)
kite.set_access_token(access_token)

In [ ]:
kite.access_token

In [ ]:
kite.profile()

In [ ]:
cancel_order('CIPLA')

In [ ]:
orders_df = pd.DataFrame(kite.orders())
orders_df[['order_id','status','tradingsymbol','transaction_type','quantity','tag','price']]

In [ ]:
live_cache.getValue()

In [ ]:
position_df = pd.DataFrame(kite.positions()['net'])
position_df

In [ ]:
pd.DataFrame(kite.positions()['day'])

In [ ]:
orders_df = pd.DataFrame()
for msg in live_cache.xrange('notificationQueuelivenew'):
    tmp_df = pd.DataFrame.from_dict(json.loads(msg[1]['data']), orient='index').transpose()
    orders_df = orders_df.append(tmp_df)
    #print(msg)
orders_df[['order_id', 'status', 'tradingsymbol','order_type','transaction_type','quantity', 'average_price']]

In [ ]:
msgs_q = live_cache.xread({'notificationQueuelivenew':'0'}, block=2000, count=5000)

In [ ]:
last_msg_id = '0'

while(True):
    if manager.abort == True:
        break;
    msgs_q = live_cache.xread({'notificationQueuelivenew':last_msg_id}, block=2000, count=5000)
    live_cache.xtrim(maxlen=0, approximate=False)
    for msg in msgs_q[0][1]:
        last_msg_id = msg[0]
        #print(msg[1]['data'])
        data = json.loads(msg[1]['data'])
        order_id = data['order_id']
        status = data['status']
        symbol = data['tradingsymbol']
        price = data['average_price']
        state = live_cache.getValue(symbol,'state')

        if status == 'COMPLETE':
            if state == 'PO:LONG':
                live_cache.setValue(symbol,'state','LONG')
            elif state == 'PO:SHORT':
                live_cache.setValue(symbol,'state','SHORT')
            elif state == 'SQUAREOFF:LONG' or state == 'SQUAREOFF:SHORT' :
                live_cache.setValue(symbol,'state','SCANNING')

            live_cache.setValue(symbol, 'price', price)

        elif status == 'REJECTED' or  status == 'CANCELLED' :
            if state == 'PO:LONG' or state == 'PO:SHORT':
                live_cache.setValue(symbol,'state','SCANNING')
            elif state == 'SQUAREOFF:LONG':
                live_cache.setValue(symbol,'state','LONG')
            elif state == 'SQUAREOFF:SHORT':
                live_cache.setValue(symbol,'state','SHORT')
        elif status == 'OPEN':
            print("{}:{} => {} - {}".format(symbol, order_id, status, state))

In [ ]:
orders_df.columns

In [ ]:
live_cache.getValue().columns

In [ ]:
kite.margins()

#### Starting Live Trade for a stock

In [ ]:
symbol = 'INFY'
live_cache.add(symbol)
token = int(live_cache.hmget('eq_token',symbol)[0])
live_cache.publish('live_trade_handlerlive', json.dumps({'cmd':'add','value':[token],'mode':'ltp'}))

live_cache.setValue(symbol,'qty','1')
live_cache.setValue(symbol,'SL %','0.4')
live_cache.setValue(symbol,'TP %','1')
live_cache.setValue(symbol,'algo','haikin_1_new')
live_cache.setValue(symbol,'freq','1T')
live_cache.setValue(symbol,'last_processed',datetime.now().timestamp())
live_cache.setValue(symbol,'mode','paper')

In [ ]:
symbol = 'WIPRO'
live_cache.setValue(symbol,'last_processed',datetime.now().timestamp())

In [ ]:
datetime.now().timestamp()

In [ ]:
symbol = 'INFY'
live_cache.setValue(symbol,'mode','PAUSE')

In [ ]:
live_cache.getValue(symbol)

### Kite Ticker Handler Manipulation

In [ ]:
#live_cache.publish('live_trade_handlerlive','profile')
live_cache.publish('live_trade_handlerlive','status')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','INIT')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','CLOSE')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','START')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','profile')

In [ ]:
live_cache.smembers('live_token')

In [ ]:
values = list(map(int,live_cache.smembers('ticker_list')))
values

In [ ]:
live_cache.publish('kite_ticker_handlerlive',json.dumps({'cmd':'add','value':values,'mode':'ltp'}))

In [ ]:
live_cache.publish('kite_ticker_handlerlive',json.dumps({'cmd':'remove','value':[1510401],'mode':'quote'}))


In [ ]:
pd.DataFrame(kite.orders())

#### TickerList Manipulaitons

In [ ]:
live_cache.sadd('ticker_list','2953217')
live_cache.sadd('ticker_list','969473')

live_cache.sadd('ticker_list','1510401')

In [ ]:
values = list(map(int,live_cache.smembers('ticker_list')))
print(values)
live_cache.hmget('eq_token', values)

In [ ]:
live_cache.srem('ticker_list',1510401)

In [ ]:
live_cache.sismember('ticker_list',4268801)

In [ ]:
live_cache.delete('ticker_list')

In [ ]:
live_cache.getValue()

### Get Kite Ticker Status

In [ ]:
## Get Status of Kite
live_cache.get('Kite_Status')

### OrderHandler Monitor and Manipulation

In [ ]:
# Pause Ordering
live_cache.publish('order_handlerlive', 'pause')

In [ ]:
# Resume Ordering
live_cache.publish('order_handlerlive', 'resume')

In [ ]:
# Buy
live_cache.publish('order_handlerlive',json.dumps({'cmd':'buy','symbol':'WIPRO','price':180,'qty':1}))

In [ ]:
# Sell
live_cache.publish('order_handlerlive',json.dumps({'cmd':'sell','symbol':'WIPRO','price':190,'qty':1}))

In [ ]:
# Cancel
live_cache.publish('order_handlerlive',json.dumps({'cmd':'cancel','symbol':'WIPRO'}))

In [ ]:
live_cache.publish('order_handlerlive',json.dumps({'cmd':'getOrders'}))

In [ ]:
## Cancel All the orders
live_cache.publish('order_handlerlive',json.dumps({'cmd':'cancelAll'}))

In [ ]:
#from lib.kite_wrapper_lib import *
cancel_order(kite,['CIPLA'])

In [ ]:
buy_limit('WIPRO',188.5,1)

In [ ]:
sell_limit('WIPRO',188.7,1)

In [ ]:
live_cache.getOHLC('WIPRO').tail(10)

In [ ]:
kite.cancel_order(kite.VARIETY_REGULAR, 200511002924441, parent_order_id=None)

In [ ]:
kite.convert_position(kite.EXCHANGE_NSE, 'WIPRO', transaction_type, position_type, quantity, old_product, new_product)


In [ ]:
kite.modify_order(kite.VARIETY_REGULAR, 200511003019322, parent_order_id=None, quantity=None, price=187.4, order_type=None, trigger_price=None, validity=None, disclosed_quantity=None)


In [ ]:
msg = {'data': '[{"tradable": true, "mode": "quote", "instrument_token": 969473, "last_price": 184.7, "last_quantity": 5, "average_price": 187.07, "volume": 4393242, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 185.0, "high": 189.95, "low": 184.1, "close": 184.0}, "change": 0.38043478260868946}, {"tradable": true, "mode": "quote", "instrument_token": 2953217, "last_price": 1935.3, "last_quantity": 25, "average_price": 1939.47, "volume": 2449395, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 1909.0, "high": 1951.4, "low": 1909.0, "close": 1893.4}, "change": 2.2129502482306886}]'}

In [ ]:
msg['data']

In [ ]:
val = json.loads(msg['data'])

##### Reset state for tradejob

In [ ]:
live_cache.setValue('WIPRO','state','SCANNING')

### Live Cache State Monitor and manipulation

In [ ]:
live_cache.getValue()

In [ ]:
#Modify cache table
stock_key = 'CIPLA'
#live_cache.add(stock_key, True) #TODO: While adding stock to table use True
live_cache.setValue(stock_key, 'algo', 'haikin_1_new')
live_cache.setValue(stock_key, 'freq', '1T')
live_cache.setValue(stock_key, 'qty', 1)
live_cache.setValue(stock_key, 'SL %', 1)
live_cache.setValue(stock_key, 'TP %', 1)
live_cache.setValue(stock_key, 'P&L', 0)
live_cache.setValue(stock_key, 'Total P&L', 0)
live_cache.setValue(stock_key, 'price', 0)
live_cache.setValue(stock_key, 'hdf_freq', 'minute')
live_cache.getValue()

In [ ]:
live_cache.setValue('WIPRO', 'mode', 'paper')

In [ ]:
#live_cache.remove('AXISBANK')
live_cache.getValue()

In [ ]:
live_cache.remove()

#### Get token for Stock symbol

In [ ]:
live_cache.hmget('eq_token',['CIPLA','INFY'])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO','AXISBANK'])

#### Peek into OHLC buffer

In [ ]:
live_cache.getOHLC('INFRATEL').tail(5)

#### msgBufferQueue Monitoring

In [ ]:
peek_msg_queue = live_cache.xrange('msgBufferQueuelive')[-1]
print(peek_msg_queue)
datetime.fromtimestamp(int(peek_msg_queue[0].split('-')[0] )/1000)

In [ ]:
datetime.fromtimestamp(1589428580968/1000)

In [ ]:
live_cache.getOHLC('INFRATEL').tail()

##### Monitor: OHLC Tick Handler

In [ ]:
val = live_cache.get('last_id_msg')
print(val)
datetime.fromtimestamp(int(val.split('-')[0] )/1000)

In [ ]:
# Timestamp to ID
timestamp_to_id = lambda x:str(int(float(x)*1000))+'-'+str(0)
id_val = timestamp_to_id(live_cache.getValue('WIPRO','last_processed'))
print(id_val)
live_cache.xread({'msgBufferQueuelive':id_val}, count=1)

In [ ]:
# Peek into the last msg processed by tick handler
print(live_cache.getValue('WIPRO','last_processed'))
datetime.fromtimestamp(float(live_cache.getValue('WIPRO','last_processed')))

In [ ]:
## Ticker job monitoring
live_cache.get('tick_count')

In [ ]:
datetime.fromtimestamp(1589535456664/1000)

In [ ]:
msgBufferDump = live_cache.xrange('msgBufferQueuelive')

In [ ]:
fp = open('log/ticker_log.log',mode='w')

In [ ]:
fp.write(json.dumps(msgBufferDump))

### Update Redis with Instrument list

##### Download fresh data and store in cache

In [ ]:
instruments_df = pd.DataFrame(kite.instruments())

In [ ]:
instruments_df.to_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df.columns

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),'instrument_token'].values

##### Use cache

In [ ]:
instruments_df = pd.read_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),:].values

eq_nse.set_index('instrument_token')

In [ ]:
live_cache.delete('eq_token')

In [ ]:
eq_nse.head().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('tradingsymbol').transpose().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('instrument_token').transpose().to_dict(orient='records')[0]

##### Update redis with tokens

In [ ]:
live_cache.hmset('eq_token',eq_nse.set_index('instrument_token').transpose().to_dict(orient='records')[0])
live_cache.hmset('eq_token',eq_nse.set_index('tradingsymbol').transpose().to_dict(orient='records')[0])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO'])

In [ ]:
live_cache.hgetall('eq_token')

In [ ]:
live_cache.hmget('eq_token',[2953217, 969473])